In [2]:
#QUERY_URL = "//div[@class='separator']//a/@href"
# QUERY_URL = "//h2[@class='inner-item__title']//a/@href"

In [1]:
import random
import requests
from requests_html import HTMLSession
from requests_html import HTML
import os
import json
import validators
from time import sleep
from functools import partial

In [2]:
from os import mkdir, listdir
import pandas as pd
from time import sleep
from os.path import join, isfile, isdir, exists, abspath
from tqdm import tqdm, trange
import re
from functools import reduce
from PIL import Image

In [3]:
class Crawler:

    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    def __init__(self, query_url):

        # ruta XPATH hacia los enlaces que queremos recopilar
        self.query_extract_all_href = query_url

        # variable que permite listar todos los enlaces recopilado
        self.all_json_to_send = []
        

    def new_session_get(self):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        return partial(session.get, headers=header)

    def start_request(self, url):
        session_get = self.new_session_get()
        response = session_get(url)
        return response

    def parse(self, response):
        self.all_json_to_send = []
        all_extracted_href = response.html.xpath(self.query_extract_all_href)
        for href in all_extracted_href:
            json_to_send = dict()
            #path = "https://www.hindustantimes.com"
            # json_to_send["url"]=path+href
            json_to_send["url"] = href
            self.all_json_to_send.append(json_to_send)


# armando el scraper

In [4]:
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
crawler = Crawler(QUERY_URL)
noticia_a = 'https://www.cnnchile.com/pais/the-strokes-anuncia-disco-2020_20200101/'
response = crawler.start_request(noticia_a)
noticia_dict = {'url': noticia_a}
noticia_dict

{'url': 'https://www.cnnchile.com/pais/the-strokes-anuncia-disco-2020_20200101/'}

In [10]:
QUERY_TITLE = "//h1[@class='main-single-header__title']/text()"
noticia_dict['title'] = response.html.xpath(QUERY_TITLE)[0]
response.html.xpath(QUERY_TITLE)[0]

'Muere la reconocida escritora española Almudena Grandes a los 61 años'

In [13]:
QUERY_BAJADA = "//div[@class='main-single-header__excerpt']//p/text()"
noticia_dict['text'] = re.sub(
    u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
noticia_dict['text']

'La escritora murió a los 61 años a causa del cáncer que dio a conocer en una  columna publicada en el diario El País. El presidente del gobierno español, Pedro Sánchez y el artista Alejandro Sanz lamentaron su partida a través de redes sociales. '

In [14]:
QUERY_CONTENT = "//div[@class='main-single-body__content']//p"
new_lines = response.html.xpath(QUERY_CONTENT)[0].text.splitlines()
re.match(r'^(.*)(– )(.*)', new_lines[0]).groups()

('(CNN Español) ',
 '– ',
 'La reconocida escritora española Almudena Grandes falleció este sábado, así lo confirmó El País, diario en el cual Grandes se desempeñaba como columnista.')

In [15]:
lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
new_lines = [re.sub(u'\xa0', u' ', line)
             for line in lines if not line.startswith('Lee también:')]
new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
noticia_dict['text'] += ' '.join(new_lines)
noticia_dict['text']

# crawling in my skiiiiin

In [4]:
def get_urls(seed_crawl, query_url, limit, start=0, leave=False, extra_validation=None):
    crawler = Crawler(query_url)
    url_noticias = []
    for page in trange(start, limit, desc='url noticias', leave=leave):
        response = crawler.start_request(seed_crawl + str(page))
        crawler.parse(response)
        if extra_validation is not None:
            noticias = [obj['url']
                        for obj in crawler.all_json_to_send if extra_validation in obj['url']]
        else:
            noticias = [obj['url'] for obj in crawler.all_json_to_send]
        url_noticias += noticias

    print('total url noticias recopiladas: ', len(url_noticias))
    return url_noticias

In [5]:
QUERY_TITLE = "//h1[@class='main-single-header__title']/text()"
QUERY_BAJADA = "//div[@class='main-single-header__excerpt']//p/text()"
QUERY_CONTENT = "//div[@class='main-single-body__content']//p"

In [6]:
def scrap_cnn_news(urls, starting_id, category, sleep_time=0):
    crawler = Crawler('')
    noticias_dicts = []
    for i, url in enumerate(tqdm(urls)):
    # algunas noticias llevan a otros sitios (en planeta pasa)
        if re.match(r'^.*//www.(\w+)\..*', url).groups()[0] == 'cnnchile':
            response = crawler.start_request(url)
            _dict = {'id': i + starting_id,
                     'url': url}
            try:
                _dict['title'] = response.html.xpath(QUERY_TITLE)[0]
                _dict['text'] = re.sub(u'\xa0', u' ', response.html.xpath(QUERY_BAJADA)[0])
                _dict['category'] = category
                lines = [line.text for line in response.html.xpath(QUERY_CONTENT)]
                new_lines = [re.sub(u'\xa0', u' ', line)
                             for line in lines if not line.startswith('Lee también:')]
                new_lines[0] = re.match(r'^(.*– )?(.*)', new_lines[0]).groups()[1]
                _dict['text'] += ' '.join(new_lines)
                noticias_dicts.append(_dict)
            except IndexError:
                print(i, url)
            sleep(sleep_time)
    return noticias_dicts

In [13]:
SEED_CRAWL = 'https://www.cnnchile.com/category/cultura/page/'
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
STARTING_ID_CULTURA = 18001000

In [14]:
START = 71 # NOTICIAS DE AGOSTO 2020 HACIA ATRAS
LIMIT = 118 # ULTIMA NOTICIA CON FECHA 31-12-2019 18:05
url_noticias_cultura = get_urls(SEED_CRAWL, QUERY_URL,
                                limit=LIMIT, start=71,
                                extra_validation='cultura')

url noticias: 100%|█████████████████████████████| 47/47 [01:04<00:00,  1.38s/it]

total url noticias recopiladas:  545


In [23]:
noticias_cultura = scrap_cnn_news(url_noticias_cultura,
                                  STARTING_ID_CULTURA,
                                  '6-Cultura_y_Artes')

100%|█████████████████████████████████████████| 545/545 [06:06<00:00,  1.49it/s]


In [24]:
df = pd.DataFrame(noticias_cultura)
df

,id,url,title,text,category
0,18001000,https://www.cnnchile.com/cultura/hollywood-llo...,“Un auténtico caballero en la pantalla y en la...,"Estudios cinematográficos, directores, compañe...",6-Cultura_y_Artes
1,18001001,https://www.cnnchile.com/cultura/beto-cuevas-e...,Beto Cuevas sobre el mundo post pandemia: “Nos...,"En conversación con CNN Chile y CHV Noticias, ...",6-Cultura_y_Artes
2,18001002,https://www.cnnchile.com/cultura/chadwick-bose...,"Murió Chadwick Boseman, protagonista de “Black...",De acuerdo a la información publicada por las ...,6-Cultura_y_Artes
3,18001003,https://www.cnnchile.com/cultura/yorka-lanzami...,Una celebración de 7 años de carrera: Yorka la...,"A un año de su realización, el proyecto de las...",6-Cultura_y_Artes
4,18001004,https://www.cnnchile.com/cultura/fotografo-chi...,Fotógrafo chileno es destacado en los prestigi...,"""Chile resiste"" se titula la obra de Javier Ve...",6-Cultura_y_Artes
...,...,...,...,...,...
540,18001540,https://www.cnnchile.com/cultura/concierto-gia...,Gianluca anuncia lanzamiento en vivo de “Yin Y...,El cantante chileno presentará su primer álbum...,6-Cultura_y_Artes
541,18001541,https://www.cnnchile.com/cultura/hatsune-miku-...,"Hatsune Miku, la estrella virtual del pop, se ...",Habiendo teloneado presentaciones de artistas ...,6-Cultura_y_Artes
542,18001542,https://www.cnnchile.com/cultura/senado-aprueb...,Senado aprueba “Ley Teloneros” para fomentar p...,"""Hace 12 que este proyecto iba avanzando. Hubo...",6-Cultura_y_Artes
543,18001543,https://www.cnnchile.com/cultura/el-gran-2019-...,"Concierto en el Movistar Arena, intervención e...","Triunfó en los premios Grammy, su gira para pr...",6-Cultura_y_Artes


In [26]:
df_cultura = pd.read_csv('./datasets/extra_cultura.csv') 
df_cultura = pd.concat([df, df_cultura])
df_cultura['content'] = df_cultura[['title', 'text']].agg('. '.join, axis=1)
df_cultura['category'] = '6-Cultura_y_Artes'
df_cultura.drop_duplicates(subset=['title'], keep='last', inplace=True)
len(df_cultura)

901

In [27]:
df_cultura.to_csv('./datasets/extra_cultura_cnn_2020-2021.csv', index=False)

# planeta

In [8]:
# trabajo manual, buscar tags que puedan estar relacionados con ecologia y planeta
tags_ecologia_planeta = [('animales', 15),            # total 19
                         ('maltrato-animal', 6),     # total 6
                         ('especies', 2),            # total 2
                         ('crisis-climatica', 4),    # total 4
                         ('cambio-climatico', 15),    # total 34
                         ('emergencia-climatica', 3),# total 3
                         ('vida-silvestre', 1),      # total 1
                         ('humedales', 1),            # total 1
                         ('fauna', 5),               # total 5
                         ('medioambiente', 8),       # total 8
                         ('medio-ambiente', 15),      # total 22
                         ]

In [9]:
seed = f'https://www.cnnchile.com/tag/animales/page/'
get_urls(seed, QUERY_URL, limit=10)

total url noticias recopiladas:  120


['https://www.cnnchile.com/pais/camara-diputados-aprueba-prohibir-testeo-animales_20211201/',
 'https://www.cnnchile.com/elecciones2021/admisible-querella-vaca-rayada-propaganda-kast_20211125/',
 'https://www.cnnchile.com/mundo/francia-nueva-ley-prohibe-venta-perros-gatos-tiendas-de-animales_20211119/',
 'https://www.cnnchile.com/tendencias/reportaje-mitos-ser-vegano-es-mas-caro_20211101/',
 'https://www.cnnchile.com/pais/dia-libre-licencia-luto-mascotas-proyecto-de-ley_20211013/',
 'https://www.cnnchile.com/mundo/adolescentes-acusados-matar-14-canguros-crias_20211012/',
 'https://www.cnnchile.com/pais/proyecto-estandares-bienestar-industria-acuicola_20210924/',
 'https://www.cnnchile.com/pais/proyecto-para-frenar-rodeo-municipios_20210901/',
 'https://www.cnnchile.com/pais/recurso-proteccion-huevos-gallinas-libres_20210828/',
 'https://www.cnnchile.com/mundo/prohiben-mujer-visitar-chimpance-amo-ese-animal_20210825/',
 'https://www.cnnchile.com/mundo/tortuga-gigante-vegetariana-comerse

In [10]:
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
url_noticias_planeta = []
for (tag_name, num_pages) in tags_ecologia_planeta:
    seed = f'https://www.cnnchile.com/tag/{tag_name}/page/'
    url_noticias_planeta += get_urls(seed, QUERY_URL, limit=num_pages, leave=False)
len(url_noticias_planeta)

total url noticias recopiladas:  180


total url noticias recopiladas:  72


total url noticias recopiladas:  24


total url noticias recopiladas:  48


total url noticias recopiladas:  180


...

total url noticias recopiladas:  5


total url noticias recopiladas:  8


total url noticias recopiladas:  96


total url noticias recopiladas:  180


832

In [11]:
STARTING_ID_PLANETA = 19000000
noticias_planeta = scrap_cnn_news(url_noticias_planeta,
                                  STARTING_ID_PLANETA,
                                  '8-Ecología_y_Planeta',
                                 sleep_time=3)




 92%|█████████████████████████████████████▊   | 767/832 [46:42<04:18,  3.97s/it]

767 https://www.cnnchile.com/cop25/futuro-360-en-cnn-chile-capitulo-55-cop25-desde-la-ciudadania_20190717/


 93%|██████████████████████████████████████▏  | 776/832 [47:49<06:01,  6.46s/it]

776 https://www.cnnchile.com/cop25/greta-thunberg-vendria-a-chile-para-la-cop25-y-no-lo-haria-en-avion_20190602/


 94%|██████████████████████████████████████▋  | 785/832 [48:36<04:06,  5.25s/it]

785 https://www.cnnchile.com/cop25/el-evento-mas-grande-desde-el-mundial-del-62-onu-ratifica-que-cop25-se-organizara-en-chile_20190307/


100%|█████████████████████████████████████████| 832/832 [52:10<00:00,  3.76s/it]


In [28]:
df = pd.DataFrame(noticias_planeta)
df

,id,url,title,text,category
0,19000000,https://www.cnnchile.com/pais/camara-diputados...,Cámara aprueba proyecto que prohíbe el testeo ...,"La iniciativa, que pasará a su segundo trámite...",8-Ecología_y_Planeta
1,19000001,https://www.cnnchile.com/elecciones2021/admisi...,Declaran admisible querella por vaca rayada co...,La acción legal fue presentada por la Fundació...,8-Ecología_y_Planeta
2,19000002,https://www.cnnchile.com/mundo/francia-nueva-l...,Francia: Nueva ley prohíbe la venta de perros ...,La normativa también eleva la pena máxima por ...,8-Ecología_y_Planeta
3,19000003,https://www.cnnchile.com/tendencias/reportaje-...,¿Es más caro ser vegano? Mitos y realidades en...,El estilo de vida que aboga por disminuir al m...,8-Ecología_y_Planeta
4,19000004,https://www.cnnchile.com/pais/dia-libre-licenc...,Proyecto de ley busca otorgar licencia por lut...,La iniciativa del diputado y médico veterinari...,8-Ecología_y_Planeta
...,...,...,...,...,...
814,19000827,https://www.cnnchile.com/pais/comite-de-minist...,Comité de Ministros aprueba por unanimidad pro...,Nido de Águilas y Embalse Digua fueron aprobad...,8-Ecología_y_Planeta
815,19000828,https://www.cnnchile.com/editorial/cnn-prime-b...,CNN Prime: Bachelet verde,Así comenzó CNN Prime este 21 de septiembre de...,8-Ecología_y_Planeta
816,19000829,https://www.cnnchile.com/pais/empresa-china-pr...,Empresa china propone a Corfo fabricar vehícul...,Pretenden avanzar en el desarrollo y puesta en...,8-Ecología_y_Planeta
817,19000830,https://www.cnnchile.com/mundo/dos-hipopotamos...,Dos hipopótamos actuaron como héroes y salvaro...,"Sucedió en el Parque Nacional Kruger, en Sudáf...",8-Ecología_y_Planeta


In [29]:
df_planeta = df
df_planeta['content'] = df[['title', 'text']].agg('. '.join, axis=1)
df_planeta.drop_duplicates(subset=['title'], keep='last', inplace=True)
df_planeta.reset_index(inplace=True)
df_planeta

,index,id,url,title,text,category,content
0,12,19000012,https://www.cnnchile.com/pais/camara-diputados...,Cámara aprueba proyecto que prohíbe el testeo ...,"La iniciativa, que pasará a su segundo trámite...",8-Ecología_y_Planeta,Cámara aprueba proyecto que prohíbe el testeo ...
1,13,19000013,https://www.cnnchile.com/elecciones2021/admisi...,Declaran admisible querella por vaca rayada co...,La acción legal fue presentada por la Fundació...,8-Ecología_y_Planeta,Declaran admisible querella por vaca rayada co...
2,15,19000015,https://www.cnnchile.com/tendencias/reportaje-...,¿Es más caro ser vegano? Mitos y realidades en...,El estilo de vida que aboga por disminuir al m...,8-Ecología_y_Planeta,¿Es más caro ser vegano? Mitos y realidades en...
3,16,19000016,https://www.cnnchile.com/pais/dia-libre-licenc...,Proyecto de ley busca otorgar licencia por lut...,La iniciativa del diputado y médico veterinari...,8-Ecología_y_Planeta,Proyecto de ley busca otorgar licencia por lut...
4,17,19000017,https://www.cnnchile.com/mundo/adolescentes-ac...,Adolescentes son acusados de matar a 14 cangur...,Los dos jóvenes de 17 años deberán comparecer ...,8-Ecología_y_Planeta,Adolescentes son acusados de matar a 14 cangur...
...,...,...,...,...,...,...,...
646,814,19000827,https://www.cnnchile.com/pais/comite-de-minist...,Comité de Ministros aprueba por unanimidad pro...,Nido de Águilas y Embalse Digua fueron aprobad...,8-Ecología_y_Planeta,Comité de Ministros aprueba por unanimidad pro...
647,815,19000828,https://www.cnnchile.com/editorial/cnn-prime-b...,CNN Prime: Bachelet verde,Así comenzó CNN Prime este 21 de septiembre de...,8-Ecología_y_Planeta,CNN Prime: Bachelet verde. Así comenzó CNN Pri...
648,816,19000829,https://www.cnnchile.com/pais/empresa-china-pr...,Empresa china propone a Corfo fabricar vehícul...,Pretenden avanzar en el desarrollo y puesta en...,8-Ecología_y_Planeta,Empresa china propone a Corfo fabricar vehícul...
649,817,19000830,https://www.cnnchile.com/mundo/dos-hipopotamos...,Dos hipopótamos actuaron como héroes y salvaro...,"Sucedió en el Parque Nacional Kruger, en Sudáf...",8-Ecología_y_Planeta,Dos hipopótamos actuaron como héroes y salvaro...


In [30]:
df_planeta.to_csv('./datasets/extra_planeta_cnn.csv', index=False)

In [ ]:
incendio
bombero
choque
accidente


# Catástrofes_y_Accidentes

In [8]:
# trabajo manual, buscar tags que puedan estar relacionados con catastrofes y accidentes
tags_catastrofes = [('atropello', 6),            # total 6
                         ('accidente', 20),     # total 30
                         ('incendio', 1),            # total 1
                         ]

In [9]:
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
url_noticias_catastrofes = []
for (tag_name, num_pages) in tags_catastrofes:
    seed = f'https://www.cnnchile.com/tag/{tag_name}/page/'
    url_noticias_catastrofes += get_urls(seed, QUERY_URL, limit=num_pages,
                                     leave=False, extra_validation='/pais/')
len(url_noticias_catastrofes)

total url noticias recopiladas:  40


total url noticias recopiladas:  104


total url noticias recopiladas:  7


151

In [ ]:
SEED_CRAWL = 'https://www.cnnchile.com/category/pais/page/'
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"


In [14]:
QUERY_URL = "//h2[@class='inner-item__title']//a/@href"
url_noticias_pais = []

seed = 'https://www.cnnchile.com/category/pais/page/'
url_noticias_pais = get_urls(seed, QUERY_URL, limit=380)


total url noticias recopiladas:  4560


In [16]:
words = ['incendio', 'bombero', 'accidente', 'choque', 'tragedia']
urls = [ link  for link in url_noticias_pais if any(word in link for word in words)]
len(urls)

65

In [17]:
_url_catastrofes = url_noticias_catastrofes + urls

In [19]:
STARTING_ID_CATASTROFES = 20000000
noticias_catastrofes = scrap_cnn_news(_url_catastrofes,
                                  STARTING_ID_CATASTROFES,
                                  '5-Catástrofes_y_Accidentes',
                                  sleep_time=3)

100%|█████████████████████████████████████████| 216/216 [13:13<00:00,  3.67s/it]


In [46]:
df = pd.DataFrame(noticias_catastrofes)
df

,id,url,title,text,category
0,20000000,https://www.cnnchile.com/pais/mujer-fallecio-a...,Una mujer falleció tras ser atropellada por un...,Los antecedentes del accidente fueron entregad...,5-Catástrofes_y_Accidentes
1,20000001,https://www.cnnchile.com/pais/funcionario-ejer...,Funcionario del Ejército sufrió amputación de ...,"El ministro del Interior, Rodrigo Delgado, cal...",5-Catástrofes_y_Accidentes
2,20000002,https://www.cnnchile.com/pais/cidh-oficio-chil...,CIDH envía oficio en que pide a Chile nuevo pr...,El organismo internacional dio un plazo de un ...,5-Catástrofes_y_Accidentes
3,20000003,https://www.cnnchile.com/pais/hombre-muere-atr...,Hombre murió tras ser atropellado por camión q...,"Según información preliminar, se trata de un m...",5-Catástrofes_y_Accidentes
4,20000004,https://www.cnnchile.com/pais/vehiculo-policia...,Vehículo de Carabineros atropelló a niño de 4 ...,El menor fue embestido por un furgón que circu...,5-Catástrofes_y_Accidentes
...,...,...,...,...,...
211,20000211,https://www.cnnchile.com/pais/imagenes-incendi...,Enorme nube de humo cubre gran parte de la reg...,Al menos seis viviendas ya se vieron afectadas...,5-Catástrofes_y_Accidentes
212,20000212,https://www.cnnchile.com/pais/incendio-foresta...,Incendio forestal en Quilpué: Onemi ordenó la ...,"Ricardo Toro, director del organismo, dijo que...",5-Catástrofes_y_Accidentes
213,20000213,https://www.cnnchile.com/pais/incendio-foresta...,Incendio forestal en Valparaíso y Quilpué ya c...,De acuerdo a los antecedentes entregados por C...,5-Catástrofes_y_Accidentes
214,20000214,https://www.cnnchile.com/pais/alerta-roja-pure...,Alerta Roja en Purén: Incendio forestal amenaz...,"Hasta el momento, 280 hectáreas han sido consu...",5-Catástrofes_y_Accidentes


In [47]:
df_catastrofes = df
df_catastrofes['content'] = df[['title', 'text']].agg('. '.join, axis=1)
df_catastrofes.drop_duplicates(subset=['title'], keep='last', inplace=True)
#df_catastrofes.reset_index(inplace=True)
df_catastrofes

,id,url,title,text,category,content
8,20000008,https://www.cnnchile.com/pais/mujer-fallecio-a...,Una mujer falleció tras ser atropellada por un...,Los antecedentes del accidente fueron entregad...,5-Catástrofes_y_Accidentes,Una mujer falleció tras ser atropellada por un...
9,20000009,https://www.cnnchile.com/pais/funcionario-ejer...,Funcionario del Ejército sufrió amputación de ...,"El ministro del Interior, Rodrigo Delgado, cal...",5-Catástrofes_y_Accidentes,Funcionario del Ejército sufrió amputación de ...
10,20000010,https://www.cnnchile.com/pais/cidh-oficio-chil...,CIDH envía oficio en que pide a Chile nuevo pr...,El organismo internacional dio un plazo de un ...,5-Catástrofes_y_Accidentes,CIDH envía oficio en que pide a Chile nuevo pr...
11,20000011,https://www.cnnchile.com/pais/hombre-muere-atr...,Hombre murió tras ser atropellado por camión q...,"Según información preliminar, se trata de un m...",5-Catástrofes_y_Accidentes,Hombre murió tras ser atropellado por camión q...
13,20000013,https://www.cnnchile.com/pais/prision-preventi...,En prisión preventiva quedó el delantero de Au...,El delantero Rodrigo Holgado transitaba en su ...,5-Catástrofes_y_Accidentes,En prisión preventiva quedó el delantero de Au...
...,...,...,...,...,...,...
211,20000211,https://www.cnnchile.com/pais/imagenes-incendi...,Enorme nube de humo cubre gran parte de la reg...,Al menos seis viviendas ya se vieron afectadas...,5-Catástrofes_y_Accidentes,Enorme nube de humo cubre gran parte de la reg...
212,20000212,https://www.cnnchile.com/pais/incendio-foresta...,Incendio forestal en Quilpué: Onemi ordenó la ...,"Ricardo Toro, director del organismo, dijo que...",5-Catástrofes_y_Accidentes,Incendio forestal en Quilpué: Onemi ordenó la ...
213,20000213,https://www.cnnchile.com/pais/incendio-foresta...,Incendio forestal en Valparaíso y Quilpué ya c...,De acuerdo a los antecedentes entregados por C...,5-Catástrofes_y_Accidentes,Incendio forestal en Valparaíso y Quilpué ya c...
214,20000214,https://www.cnnchile.com/pais/alerta-roja-pure...,Alerta Roja en Purén: Incendio forestal amenaz...,"Hasta el momento, 280 hectáreas han sido consu...",5-Catástrofes_y_Accidentes,Alerta Roja en Purén: Incendio forestal amenaz...


In [48]:
df_catastrofes.to_csv('./datasets/extra_catastrofes_cnn.csv', index=False)

In [49]:
df_catastrofes = pd.read_csv('./datasets/extra_catastrofes_cnn.csv')
df_planeta = pd.read_csv('./datasets/extra_planeta_cnn.csv')
df_cultura = pd.read_csv('./datasets/extra_cultura_cnn_2020-2021.csv')
df_paso3 = pd.read_csv('./datasets/dataset_paso_3.csv')

In [56]:
df_final = pd.concat([df_paso3, df_cultura, df_planeta, df_catastrofes])
df_final.drop_duplicates(subset=['title'], keep='last', inplace=True)
df_final.reset_index(inplace=True)
df_final.drop(columns=['level_0'], inplace=True)
df_final.to_csv('./datasets/dataset_paso3d96.csv', index=False)

In [58]:
grouped = df_final.groupby(by='category').size().sort_values()
grouped

category
5-Catástrofes_y_Accidentes     1430
8-Ecología_y_Planeta           1452
6-Cultura_y_Artes              1678
2-Economía                     2635
10-Salud                       2683
4-Ciencias_y_Tecnología        4772
7-Deporte                      6702
9-Crimen_delitos_y_Justicia    8212
1-Mundo                        8586
3-Política_y_Conflictos        8951
dtype: int64

In [59]:
SAMPLE_SIZE = grouped[0]
SAMPLE_SIZE

1430

In [62]:
RANDOM_STATE = 5150
new_df = df_final.groupby('category').sample(n=SAMPLE_SIZE,
                                             random_state=RANDOM_STATE)
new_df.reset_index(drop=True, inplace=True)
new_df.to_csv(f'./datasets/ds_balanced_{SAMPLE_SIZE}_rs{RANDOM_STATE}.csv', index=False)

In [81]:
df_final[~df_final['category'].isin(_cols)]

,id,title,content,category,url,text,index
0,13115,Vacunación en Chile empezaría primer trimestre...,Vacunación en Chile empezaría primer trimestre...,10-Salud,NaN,NaN,NaN
1,13183,"Sernapesca: ""Viabilidad del virus de infectar ...","Sernapesca: ""Viabilidad del virus de infectar ...",10-Salud,NaN,NaN,NaN
2,614680,"Presidente de México asegura que la pandemia ""...","Presidente de México asegura que la pandemia ""...",1-Mundo,NaN,NaN,NaN
3,614690,Gobierno de Uruguay endurece medidas por aumen...,Gobierno de Uruguay endurece medidas por aumen...,1-Mundo,NaN,NaN,NaN
4,614700,Presidente boliviano destituye a ministro que ...,Presidente boliviano destituye a ministro que ...,1-Mundo,NaN,NaN,NaN
...,...,...,...,...,...,...,...
46259,18000359,“Dios los Cría”: Calamaro invita a Mon Laferte...,“Dios los Cría”: Calamaro invita a Mon Laferte...,6-Cultura_y_Artes,https://www.cnnchile.com/cultura/calamaro-mon-...,"La chilena interpretará ""Tantas veces"" junto a...",NaN
46260,18000360,“Qué Weá”: Paloma Mami se reivindica como chil...,“Qué Weá”: Paloma Mami se reivindica como chil...,6-Cultura_y_Artes,https://www.cnnchile.com/cultura/paloma-mami-v...,La cantante participó en la dirección del vide...,NaN
46261,18000361,Ellen DeGeneres informó que planea terminar su...,Ellen DeGeneres informó que planea terminar su...,6-Cultura_y_Artes,https://www.cnnchile.com/cultura/ellen-degener...,"La popular animadora explicó que ""cuando eres ...",NaN
46262,18000362,Los poemas en la UCI del Dr. Glenn Hernández: ...,Los poemas en la UCI del Dr. Glenn Hernández: ...,6-Cultura_y_Artes,https://www.cnnchile.com/cultura/entrevista-do...,Casi un año antes del primer muerto por la pan...,NaN


In [84]:
SAMPLE_SIZE = 1678
_cols = ['5-Catástrofes_y_Accidentes', '8-Ecología_y_Planeta']
new_df = df_final[~df_final['category'].isin(_cols)].groupby('category').sample(n=SAMPLE_SIZE,
                                             random_state=RANDOM_STATE,
                                            )

xd =  df_final[df_final['category'].isin(_cols)].groupby('category').sample(n=SAMPLE_SIZE,
                                                 random_state=RANDOM_STATE,
                                                 replace=True
                                            )

In [87]:
pd.concat([new_df, xd]).to_csv(f'./datasets/ds_semibalanced_{SAMPLE_SIZE}_rs{RANDOM_STATE}.csv',
                               index=False)

In [3]:
import pandas as pd
df = pd.read_csv('./datasets/dataset_paso3d96.csv')

In [4]:
df.groupby('category').size()

category
1-Mundo                        8586
10-Salud                       2683
2-Economía                     2635
3-Política_y_Conflictos        8951
4-Ciencias_y_Tecnología        4772
5-Catástrofes_y_Accidentes     1430
6-Cultura_y_Artes              1678
7-Deporte                      6702
8-Ecología_y_Planeta           1452
9-Crimen_delitos_y_Justicia    8212
dtype: int64